In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import seaborn as sns
import pickle
from fpdf import FPDF
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

base_dir = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\Skin cancer\Dataset\skin cancer\Skin cancer ISIC The International Skin Imaging Collaboration"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

img_size = (224, 224)
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

model_save_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\skin_cancer_model.h5"
history_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\skin_cancer_history.pkl"
model.save(model_save_path)
with open(history_path, "wb") as f:
    pickle.dump(history.history, f)

test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=target_names, yticklabels=target_names, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Skin Cancer")
cm_image_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\cm_skin.png"
plt.savefig(cm_image_path)
plt.close()

y_true_bin = label_binarize(y_true, classes=np.arange(len(target_names)))
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(target_names)):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
plt.figure(figsize=(8,6))
for i in range(len(target_names)):
    plt.plot(fpr[i], tpr[i], label=f"{target_names[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0,1], [0,1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Skin Cancer")
plt.legend()
roc_image_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\roc_skin.png"
plt.savefig(roc_image_path)
plt.close()

history_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\skin_cancer_history.pkl"
with open(history_path, "rb") as f:
    history = pickle.load(f)

plt.figure()
plt.plot(history["accuracy"], label="Train Acc")
plt.plot(history["val_accuracy"], label="Val Acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
acc_image_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\acc_skin.png"
plt.savefig(acc_image_path)
plt.close()

plt.figure()
plt.plot(history["loss"], label="Train Loss")
plt.plot(history["val_loss"], label="Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
loss_image_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\loss_skin.png"
plt.savefig(loss_image_path)
plt.close()

pdf_save_path = r"C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\skin_cancer_report.pdf"
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", 'B', 16)
pdf.cell(0, 10, "Skin Cancer Detection Report", ln=True, align="C")
pdf.ln(10)
pdf.set_font("Arial", '', 12)
pdf.multi_cell(0, 6, "Objective:\nThe objective is to develop a ResNet50-based model to classify dermoscopic images of skin cancer into multiple categories.")
pdf.ln(5)
pdf.multi_cell(0, 6, f"Test Accuracy: {test_acc:.2f}")
pdf.ln(10)
pdf.set_font("Arial", 'B', 14)
pdf.cell(0, 10, "Training and Validation Accuracy", ln=True)
pdf.image(acc_image_path, w=150)
pdf.ln(10)
pdf.cell(0, 10, "Training and Validation Loss", ln=True)
pdf.image(loss_image_path, w=150)
pdf.ln(10)
pdf.cell(0, 10, "Confusion Matrix", ln=True)
pdf.image(cm_image_path, w=150)
pdf.ln(10)
pdf.cell(0, 10, "ROC Curves", ln=True)
pdf.image(roc_image_path, w=150)
pdf.output(pdf_save_path)
print("PDF Report saved at:", pdf_save_path)


Found 1795 images belonging to 9 classes.
Found 444 images belonging to 9 classes.
Found 118 images belonging to 9 classes.


C:\Users\USER\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.1993 - loss: 2.1151 - val_accuracy: 0.2140 - val_loss: 2.0291
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.2008 - loss: 2.0263 - val_accuracy: 0.2027 - val_loss: 2.0209
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.2044 - loss: 2.0225 - val_accuracy: 0.1959 - val_loss: 2.0424
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.2118 - loss: 2.0398 - val_accuracy: 0.1577 - val_loss: 2.0340
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.1974 - loss: 2.0446 - val_accuracy: 0.2072 - val_loss: 2.0461
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.1982 - loss: 2.0525 - val_accuracy: 0.2005 - val_loss: 2.0229
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.1829 - loss: 2.0501 - val_accuracy: 0.2027 - val_loss: 2.0204
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.1750 - loss: 2.0423 - val_accu

8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.1189 - loss: 2.4298
Test Accuracy: 0.18644067645072937


8/8 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step
                            precision    recall  f1-score   support

         actinic keratosis       0.00      0.00      0.00        16
      basal cell carcinoma       0.00      0.00      0.00        16
            dermatofibroma       0.00      0.00      0.00        16
                  melanoma       0.75      0.19      0.30        16
                     nevus       0.16      0.94      0.27        16
pigmented benign keratosis       0.22      0.25      0.24        16
      seborrheic keratosis       0.00      0.00      0.00         3
   squamous cell carcinoma       0.00      0.00      0.00        16
           vascular lesion       0.00      0.00      0.00         3

                  accuracy                           0.19       118
                 macro avg       0.13      0.15      0.09       118
              weighted avg       0.15      0.19      0.11       118



C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


PDF Report saved at: C:\Users\USER\OneDrive\Desktop\Developer Hub Internship\Task3\skin_cancer_report.pdf
